In [ ]:
import pandas as pd
import astropy as ap
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns

In [ ]:
import faiss

In [ ]:
infile = 'vecs/tess_ode.pkl'

In [ ]:
data = pd.read_pickle(infile)

In [ ]:
vecs = np.stack(list(data.values()))

In [ ]:
vecs = vecs/np.linalg.norm(vecs, axis=1).reshape(-1,1)

In [ ]:
i2f = dict(enumerate(list(data.keys())))

### Build DB

In [ ]:
d = vecs.shape[1]; d

In [ ]:
db = faiss.IndexFlatIP(d)

In [ ]:
db.add(vecs)

In [ ]:
db.ntotal

### Input vec

In [ ]:
indir = 'tess/16_17/z_normalized/'
filenames = [indir+x+'.npy' for x in list(data.keys())]

In [ ]:
tsne_res = pd.read_pickle('tess_tsne.pkl')

In [ ]:
df = pd.DataFrame(zip(filenames, tsne_res[:,0], tsne_res[:,1]),
                      columns = ['file','x1', 'x2'])

In [ ]:
plot_bounds = (5,7, 18, 20)

In [ ]:
s = df.loc[(df['x1'] > plot_bounds[0]) & (df['x1'] < plot_bounds[1])
       & (df['x2'] > plot_bounds[2]) & (df['x2'] < plot_bounds[3]) ]

In [ ]:
def plot_one(sample):
    file = sample['file'].values[0]
    ts = np.load(file)[1]
    plt.figure(figsize=(5,5))
    #     ax.set_title('cluster: ' + str(ixs[i]) + '  ' + 'score: ' + str(probs[i]))
    ax=sns.lineplot(x=np.arange(len(ts)), y=ts, color='k', alpha=0.8)
    ax.set(yticks=[], xticks=[])
    obj_id = file2id(file)
    fname = file.split(indir)[-1][:-4]
    return ax, ts, obj_id, fname

In [ ]:
def file2id(file):
    return file.split(indir)[-1].split('-')[2]

In [ ]:
ax, ts, obj_id, fname = plot_one(s.sample(n=1))
input_vec = data[fname]

In [ ]:
input_vec

In [ ]:
fname

In [ ]:
input_vec = input_vec/np.linalg.norm(input_vec)
input_vec = input_vec.reshape(1,-1)

In [ ]:
# %timeit db.search(input_vec, k=1000)

In [ ]:
probs, ixs = db.search(input_vec, k=4)
ixs = ixs[0]
probs = probs[0]

In [ ]:
tt = []
for i in ixs:
    f = i2f[i]
    f = indir+str(f)+'.npy'
    d = np.load(f)
    tt.append(d[1])

In [ ]:
i = 1
sns.lineplot(x=np.arange(len(tt[i])), y = tt[i])